In [1]:
import cv2
import matplotlib.pyplot as plt

from remote import validate_video, process_video, calculate_spo2, calculate_heart_rate

In [2]:
file = 'data/jmg-lamp.mp4'
s3_file = 'https://covid5-videos.s3-us-west-1.amazonaws.com/dummy_video.mp4'
video = cv2.VideoCapture(file)
validate_video(video)
video = process_video(video)

In [3]:
spo2 = calculate_spo2(video)
spo2_disc = calculate_spo2(video, discretize=True)

print(spo2)
print(spo2_disc)

97.90659908920401
normal


In [ ]:
bpm = calculate_heart_rate(video)
bpm_disc = calculate_heart_rate(video, discretize=True)

print(bpm)
print(bpm_disc)